# Morphological Segmentation

## Dependancies

- Python(>=3.6)
- os
- urllib.request
- tarfile
- python-crfsuite
- bpe
- numpy


### Reading in data

Start by running the following code. It will create a directory `.data` and download English segmentation data into the directory. 

In [1]:
import os, urllib.request, tarfile

URL = "http://mpsilfve.github.io/assets/segmentation_data.tgz"
DATADIR = ".data"
FN = "segmentation_data"
TGZ = os.path.join(DATADIR,"%s.tgz" % FN)

try:
    print("Creating",DATADIR)
    os.mkdir(DATADIR)
except FileExistsError:
    pass

print("Downloading %s.tgz into .data" % FN)
urllib.request.urlretrieve(URL,TGZ)

print("Extracting %s" % TGZ)
tf = tarfile.open(TGZ)
tf.extractall(DATADIR)

Creating .data
Extracting .data/segmentation_data.tgz


####  Reading data files into lists

In the sub-directory `segmentation_data` of the directory `.data`, there are three files: `traindata`, `devdata` and `testdata`. These files consists of two tab-separated columns:

```
vouchsafed      vouch saf ed
negative        negat ive
annotations     an not at ion s
torpedos        torpedo s
coxswain        coxswain
lofted          loft ed
...
```

The first column contains a word form and the second column contains the same word form but segmented into morphemes. Please read these files into three lists: `traindata`, `devdata` and `testdata`. Each list element should be a pair like:

```
["vouchsafed",["vouch","saf","ed"]]
```

In [5]:
traindata = []
devdata = []
testdata = []

directory = ".data/segmentation_data/"
file_names = ["traindata", "devdata", "testdata"]
files = [traindata, devdata, testdata]

for i, file in enumerate(file_names):
    f = open(directory+file)
    for line in f:
        line = line.strip("\n")
        word, morphemes = line.split("\t")
        files[i].append([word, morphemes.split(" ")])

In [6]:
# find unsegmented word forms

trainwords = [wf for wf, _ in traindata]
devwords = [wf for wf, _ in devdata]
testwords = [wf for wf, _ in testdata]

#### BIES notation

Convert segmented word forms into so called BIES (Begin-Inside-End-Singleton) format which looks like this (for the word form "bunkhouses" segmented as "bunk" "house" "s"):

```
BEGIN   INSIDE   INSIDE   END   BEGIN   INSIDE   INSIDE   INSIDE   END   SINGLE

b       u        n        k     h       o        u        s        e     s
```

The first character of each morpheme receives a `BEGIN` tag, the last one an `END` tag and the remaining characters receive an `INSIDE` tag. As a special case, morphemes consisting of a single character (like the plural "s" ending above), receive the tag `SINGLE`. 


In [13]:
BEGIN = "BEGIN"
INSIDE = "INSIDE"
END = "END" 
SINGLE = "SINGLE"

def get_bies_notation(data):
    '''
    Turn the given word and morphmes into bies notation
    '''
    notations = []
    for line in data:
        mor_notation = []
        morphemes = line[1]
        for morpheme in morphemes:
            if len(morpheme) == 1:
                mor_notation.append([morpheme, SINGLE])
            else:
                mor_notation.append([morpheme[0], BEGIN])
                for i in range(1, len(morpheme) - 1):
                    mor_notation.append([morpheme[i], INSIDE])
                mor_notation.append([morpheme[-1], END])
                
        notations.append(mor_notation)

    return notations

trainbies = get_bies_notation(traindata)
devbies = get_bies_notation(devdata)
testbies = get_bies_notation(testdata)


#### From BIES notation to morphemes

For evaluation purposes, we will need to transform BIES notation produced by our segmentation model back into segmented word forms, i.e. take the following as input:

```
[['s', 'BEGIN'], ['a', 'INSIDE'], ['l', 'INSIDE'], ['e', 'END'], ['s', 'SINGLE']]
```

And generate the following as output:

```
["sale", "s"]
```

Implement a function `unbies(data)` that takes a list of examples in BIES format as input and returns a list of pairs in the format:

```
["sales", ["sale","s"]]
```

The first element in the pair is the unsegmented word form and the second one is the segmented word form.

In [25]:
def unbies(bies_notations):
    '''
    Get the word form and morphemes from the BIES notations
    '''
    unbies_forms = []
    
    for bies_notation in bies_notations:
        chars = []
        morphemes = []
        morpheme = []
        for char in bies_notation:
            chars.append(char[0])
            if char[1] == 'BEGIN' or char[1] == 'INSIDE':
                morpheme.append(char[0])
            elif char[1] == 'END' or char[1] == 'SINGLE':
                morpheme.append(char[0])
                morphemes.append("".join(morpheme))
                morpheme = []
        
        unbies_forms.append(["".join(chars), morphemes])
        
    return unbies_forms


### BPE baseline


#### Training a BPE model

Initialize a `bpe.Encoder` model `encoder` with vocabulary size 64,000, then read training data for BPE from the file `en-ud-train.txt` in the sub-directory `segmentation_data` in the `.data` directory. Split the file into lines and call the `encoder.fit` giving the dataset as parameter.   

In [50]:
import os
from bpe.encoder import Encoder

VOCAB_SIZE=64000
PCTBPE=1
NGRAM_MAX=1000000
NGRAM_MIN=1

encoder = Encoder(VOCAB_SIZE, PCTBPE, ngram_max = NGRAM_MAX, ngram_min = NGRAM_MIN)

txt_dir = "./.data/segmentation_data/en-ud-train.txt"

f = open(txt_dir)
corpus = f.read()
encoder.fit(corpus.split('\n'))


#### Segmenting development and test data using BPE


In [45]:
bpe_segmented_dev = []
bpe_segmented_test = []

todo_words = [devwords, testwords]
bpe_segmented = [bpe_segmented_dev, bpe_segmented_test]

for i, words in enumerate(todo_words):
    for word in words:
        bpe_segmented[i].append([segment for segment in encoder.tokenize(word) if not segment.startswith("__")])
        
bpe_segmented


[[['sales'],
  ['oppress', 'or'],
  ['wi', 'pes'],
  ['bash', 'fully'],
  ['feli', 'cit', 'ous'],
  ['cere', 'brum'],
  ['cents', "'"],
  ['string', 'ing'],
  ['rever', 'tible'],
  ['cere', 'al', "'", 's'],
  ['diso', 'rient', 'ation'],
  ['muz', 'zl', 'ed'],
  ['chocolate', 's'],
  ['alter'],
  ['man', '-', 'eating'],
  ['vac', 'illa', 'tion'],
  ['je', 'll', '-', 'o', "'", 's'],
  ['ill', 'ness', "'"],
  ['methods'],
  ['tres', 'pass'],
  ['with', 'hold'],
  ['years', "'"],
  ['mitigate'],
  ['rail', 'way', "'", 's'],
  ['undertake', 'r', "'", 's'],
  ['hot', 'house'],
  ['slo', 'tted'],
  ['stra', 'fed'],
  ['blu', 'ster', "'", 's'],
  ['dinner', '-', 'service'],
  ['off', '-', 'white', "'", 's'],
  ['right'],
  ['returns', "'"],
  ['tribu', 'nal', "'", 's'],
  ['chron', 'ically'],
  ['protest', 'ants'],
  ['professors', 'hips'],
  ['jun', 'gles'],
  ['miz', 'zen'],
  ['chlorin', 'e'],
  ['sand', 'alw', 'ood'],
  ['sub', 'urban', 'ite'],
  ['win', 'ced'],
  ['freighte', 'd'],
  ['pu

### Evaluation

We will evaluate segmentation algorithms using precision, recall and f-score on segment boundaries. As an example, let's say we evaluate against the following gold standard:

```
[["hot", "dog","s"],["king","s"]]
```

and our segmentation system returned the following segmentation:

```
[["hot","dog","s"], ["k","ing","s"]]
```

The gold standard segment boundaries are:

```
[[3,6],[4]]
```

and our system gives the following segment boundaries:

```
[[3,6],[1,4]]
```

Now three of our segment boundaries are actually found in the gold standard (`3` and `6` for the first example and `4` for the second one). This gives us a precision of $P = 3/4 = 75\%$ (three of the four morpheme boundaries were found are in the gold standard) and a recall of $R = 3/3 = 100\%$ (we found two of the three token boundaries given by the gold standard). Finally, we get the f-score as $2 \cdot P \cdot R /(P+R) \approx 85.7\%$. 


In [56]:
import numpy as np

def get_boundaries(segmented_data):
    '''
    Return the morpheme boundaries given the segmented morphemes
    '''
    boundaries = []
    len_segments = [len(seg) for seg in segmented_data]

    if len(len_segments) > 1:
        boundaries.append(len_segments[0])
        for i in range(1, len(len_segments) - 1):
            boundaries.append(len_segments[i] + boundaries[i - 1])
        
    return boundaries  
        

def evaluate(sys_segmented_data,gold_segmented_data):

    correct = 0
    sys_total = 0
    gold_total = 0
    
    for i in range(len(sys_segmented_data)):
        sys_boundary = get_boundaries(sys_segmented_data[i])
        gold_boundary = get_boundaries(gold_segmented_data[i][1])
        sys_total += len(sys_boundary)
        gold_total += len(gold_boundary)
        for b in sys_boundary:
            if b in gold_boundary:
                correct += 1
    
    precision = correct/sys_total
    recall = correct/gold_total
    fscore = 2 * precision * recall / (precision + recall)

    return precision, recall, fscore
    
precision, recall, fscore = evaluate(bpe_segmented_test,testdata)
print("Results for BPE segmentation:")
print("Test set precision: %.2f, recall: %.2f, f-score: %.2f" % (precision, recall, fscore))

Results for BPE segmentation:
Test set precision: 0.46, recall: 0.48, f-score: 0.47


### Supervised morphological segmentation

Install `pycrfsuite` using pip (see [Installation](https://python-crfsuite.readthedocs.io/en/latest/)).


In [57]:
import pycrfsuite

#### Feature engineering

Implement a feature extraction function.

In [134]:
BOUNDARY="<BD>"

def char2features(example, i):
    
    features = []
    
    char_at_i = example[i][0]
    distance_to_end = str(len(example) - i)
    isalpha_at_i = str(example[i][0].isalpha())
    isdigit_at_i = str(example[i][0].isdigit())
    
    padded_example = [BOUNDARY, BOUNDARY] + [e[0] for e in example] + [BOUNDARY, BOUNDARY]
    
    left_char2 = padded_example[i]
    left_char = padded_example[i + 1]
    right_char = padded_example[i + 3]
    right_char2 = padded_example[i + 4]
    
    features.append("CHARACTER_AT_i=%s" % char_at_i)
    
    features.append("LEFT_CHARACTER=%s" % left_char)
    features.append("RIGHT_CHARACTER=%s" % right_char)
    
    features.append("LEFT_BIGRAM=%s" % left_char+char_at_i)
    features.append("RIGHT_BIGRAM=%s" % char_at_i+right_char)
    
    features.append("LEFT_SKIPGRAM=%s" % left_char2+char_at_i)
    features.append("RIGHT_SKIPGRAM=%s" % char_at_i+right_char2)
    
    features.append("LEFT_TRIGRAM=%s" % left_char2+left_char+char_at_i)
    features.append("RIGHT_TRIGRAM=%s" % char_at_i+right_char+right_char2)
    
    features.append("DISTANCE_TO_END=%s" % distance_to_end)
    
    features.append("IS_ALPHA=%s" % isalpha_at_i)
    features.append("IS_DIGIT=%s" % isdigit_at_i)

    return features

def data2features(data):
    """ Extract features for a data set in BIES format. """
    return [[char2features(example,i) for i in range(len(example))] for example in data]

def data2labels(data):
    """ Extract the tags from a data set in BIES format. """
    return [[tok[1] for tok in example] for example in data]

# Initialize the training, development and test sets for pycrfsuite.
X_train = data2features(trainbies)
y_train = data2labels(trainbies)

X_dev = data2features(devbies)
y_dev = data2labels(devbies)

X_test = data2features(testbies)
y_test = data2labels(testbies)

In [135]:
X_dev[0]

[['CHARACTER_AT_i=s',
  'LEFT_CHARACTER=<BD>',
  'RIGHT_CHARACTER=a',
  'LEFT_BIGRAM=<BD>s',
  'RIGHT_BIGRAM=sa',
  'LEFT_SKIPGRAM=<BD>s',
  'RIGHT_SKIPGRAM=sl',
  'LEFT_TRIGRAM=<BD><BD>s',
  'RIGHT_TRIGRAM=sal',
  'DISTANCE_TO_END=5',
  'IS_ALPHA=True',
  'IS_DIGIT=False'],
 ['CHARACTER_AT_i=a',
  'LEFT_CHARACTER=s',
  'RIGHT_CHARACTER=l',
  'LEFT_BIGRAM=sa',
  'RIGHT_BIGRAM=al',
  'LEFT_SKIPGRAM=<BD>a',
  'RIGHT_SKIPGRAM=ae',
  'LEFT_TRIGRAM=<BD>sa',
  'RIGHT_TRIGRAM=ale',
  'DISTANCE_TO_END=4',
  'IS_ALPHA=True',
  'IS_DIGIT=False'],
 ['CHARACTER_AT_i=l',
  'LEFT_CHARACTER=a',
  'RIGHT_CHARACTER=e',
  'LEFT_BIGRAM=al',
  'RIGHT_BIGRAM=le',
  'LEFT_SKIPGRAM=sl',
  'RIGHT_SKIPGRAM=ls',
  'LEFT_TRIGRAM=sal',
  'RIGHT_TRIGRAM=les',
  'DISTANCE_TO_END=3',
  'IS_ALPHA=True',
  'IS_DIGIT=False'],
 ['CHARACTER_AT_i=e',
  'LEFT_CHARACTER=l',
  'RIGHT_CHARACTER=s',
  'LEFT_BIGRAM=le',
  'RIGHT_BIGRAM=es',
  'LEFT_SKIPGRAM=ae',
  'RIGHT_SKIPGRAM=e<BD>',
  'LEFT_TRIGRAM=ale',
  'RIGHT_TRIGRAM=e

#### Training the segmentation system


In [136]:
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.train('segmentation.model')


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 11596
Seconds required: 0.056

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 11922.221199
Feature norm: 1.000000
Error norm: 4063.747325
Active features: 6748
Line search trials: 1
Line search step: 0.000132
Seconds required for this iteration: 0.010

***** Iteration #2 *****
Loss: 10565.671702
Feature norm: 1.237657
Error norm: 3058.402218
Active features: 7005
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.006

***** Iteration #3 *****
Loss: 6386.033361
Feature norm: 3.681648
Error norm: 2564.218418
Active features: 6103
Line search trials: 1
Line search step: 1.000000
Seconds required for this it

#### Fine-tuning and segmentation


In [137]:
def segment(data,tagger):
    tagged_data = []
    for i,example in enumerate(data2features(data)):
        tags = tagger.tag(example)
        tagged_example = [(char, tag) for (char, _), tag in zip(data[i],tags)]
        tagged_data.append(tagged_example)
    return [segmented for _,segmented in unbies(tagged_data)]
    
tagger = pycrfsuite.Tagger()
tagger.open('segmentation.model')

supervised_tokenized_dev = segment(devbies,tagger)
print("Results for supervised segmentation:")
print("Development set precision: %.2f, recall: %.2f, f-score: %.2f" % evaluate(supervised_tokenized_dev,devdata))

supervised_tokenized_test = segment(testbies,tagger)
print("Results for supervised segmentation:")
print("Test set precision: %.2f, recall: %.2f, f-score: %.2f" % evaluate(supervised_tokenized_test,testdata))

Results for supervised segmentation:
Development set precision: 0.89, recall: 0.85, f-score: 0.87
Results for supervised segmentation:
Test set precision: 0.88, recall: 0.82, f-score: 0.85
